In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
class MLP:
    """
    1 Hidden Layer MLP Class
    
    Input:  x - 1 dimensional train data
            y - 2 dimensional true values (3 rows, 1 cols)
            neurons - number of neurons in layer
            activation - sigmoid or tanh activation function (tanh, sigmoid) (default: sigmoid)
            lr - learning rate (default: 0.1)
            epochs - number of iterations (default: 1000)
    """
    # sukuriami kintamieji
    def __init__(self, x, y, neurons, activation, lr=0.1, epochs=1000):
        self.x = x
        self.y = y
        self.w1 = self.initialization(3)
        self.w2 = self.initialization(4)
        self.w3 = self.initialization(4)
        self.b1 = self.initialization(3)
        self.b2 = self.initialization(2)
        self.b3 = self.initialization(3)
        self.a1 = np.zeros(3)
        self.a2 = np.zeros(2)
        self.a3 = np.zeros(3)
        self.activation = self.selected_activation(activation)
        self.lr = lr
        self.epochs = epochs
        self.pred_y = []
    
    # kintamuju inicializacija
    def initialization(self, size):
        return np.random.rand(size)
    
    # aktyvacijos funkcijos nustatymas        
    def selected_activation(self, activation):
        activation_func = lambda activation: 'tanh' if activation == 'tanh' else 'sigmoid'
        return activation_func(activation)
    
    # aprasytos aktyvaciju funkciju formules
    def activation_function(self, z):
        if self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-z))
        else:
            return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
    
    # pirmo sluoksnio forward propagation
    def forward_propagation_layer_one(self, x):
        for i in range(3):
            z = self.w1[i] * x + self.b1[i]
            self.a1[i] = self.activation_function(z)
    
    # antro sluoksnio forward propagation        
    def forward_propagation_layer_two(self):
        temp = 0
        for i in range(2):
            z = self.w2[i] * self.a1[i]
            temp += z
        self.a2[0] = temp + self.b2[0]
        temp = 0
        for i in range(2,4):
            z = self.w2[i] * self.a1[i-1] + self.b2[i]
            temp += z
        self.a2[1] = temp + self.b2[1]
    
    # trecio sluoksnio forward propagation    
    def forward_propagation_layer_three(self):
        z = self.w3[0] * self.a2[0] + self.b3[0]
        self.a3[0] = self.activation_function(z)
        temp = 0
        for i in range(1,3):
            z = self.w3[i] * self.a2[i-1]
            temp += z
        self.a3[1] = self.activation_function(temp + self.b3[1])
        z = self.w3[3] * self.a2[1] + self.b3[2]
        self.a3[2] = self.activation_function(z)
    
    # error'o apskaiciavimas
    def error_function(self, y_true, output_layer):
        return y_true - output_layer
    
    # aprasytos aktyvaciju isvestines
    def activation_derivative(self, fi):
        if self.activation == 'sigmoid':
            return fi * (1 - fi)
        else:
            return 1 - fi**2
    
    # isejimo sluoksnio parametru atnaujinimas (back-propagation)        
    def params_update_layer_three(self, e1,e2,e3):
        self.w3[0] = self.w3[0] + self.lr * e1 * self.activation_derivative(self.a3[0]) * self.a2[0]
        self.w3[1] = self.w3[1] + self.lr * e2 * self.activation_derivative(self.a3[1]) * self.a2[0]
        self.w3[2] = self.w3[2] + self.lr * e2 * self.activation_derivative(self.a3[1]) * self.a2[1]
        self.w3[3] = self.w3[3] + self.lr * e3 * self.activation_derivative(self.a3[2]) * self.a2[1]
        self.b3[0] = self.b3[0] + self.lr * e1 * self.activation_derivative(self.a3[0])
        self.b3[1] = self.b3[1] + self.lr * e2 * self.activation_derivative(self.a3[1])
        self.b3[2] = self.b3[2] + self.lr * e2 * self.activation_derivative(self.a3[1])
        self.b3[3] = self.b3[3] + self.lr * e3 * self.activation_derivative(self.a3[2])
    
    # antrojo pasleptojo sluoksnio parametru atnaujinimas   
    def params_update_layer_two(self, e1,e2,e3):
        delta1 = self.a2[0] * (e1 * self.activation_derivative(self.a3[0]) * self.w3[1] + e2 * self.activation_derivative(self.a3[1]) * self.w3[1])
        delta2 = self.a2[1] * (e2 * self.activation_derivative(self.a3[1]) * self.w3[2] + e3 * self.activation_derivative(self.a3[2]) * self.w3[3])
        self.w2[0] = self.w2[0] + self.lr * delta1 * self.a1[0]
        self.w2[1] = self.w2[1] + self.lr * delta1 * self.a1[1]
        self.w2[2] = self.w2[2] + self.lr * delta2 * self.a1[1]
        self.w2[3] = self.w2[3] + self.lr * delta2 * self.a1[2]
        self.b2[0] = self.b2[0] + self.lr * delta1
        self.b2[1] = self.b2[1] + self.lr * delta1
        self.b2[2] = self.b2[2] + self.lr * delta2
        self.b2[3] = self.b2[3] + self.lr * delta2
        return delta1,delta2
    
    # pirmojo pasleptojo sluoksnio parametru atnaujinimas
    def params_update_layer_one(self, delta11,delta22,x):
        delta1 = self.a1[0] * delta11
        delta2 = self.a1[1] * (delta11 + delta22)
        delta3 = self.a1[2] * delta22
        self.w1[0] = self.w1[0] + self.lr * delta1 * x
        self.w1[1] = self.w1[1] + self.lr * delta2 * x
        self.w1[2] = self.w1[2] + self.lr * delta3 * x
        self.b1[0] = self.b1[0] + self.lr * delta1
        self.b1[1] = self.b1[1] + self.lr * delta2
        self.b1[2] = self.b1[2] + self.lr * delta3
    
    # treniravimo etapas. Treniruojasi nurodyta skaiciu epochu        
    def train(self):
        for j in range(self.epochs):
            for i in range(len(self.x)):
                self.forward_propagation_layer_one(self.x[i])
                self.forward_propagation_layer_two()
                self.forward_propagation_layer_three()
                
                e1 = self.error_function(self.y[i][0], self.a3[0])
                e2 = self.error_function(self.y[i][1], self.a3[1])
                e3 = self.error_function(self.y[i][2], self.a3[2])
                
                self.params_update_layer_three(e1,e2,e3)
                delta1,delta2 = self.params_update_layer_two(e1,e2,e3)
                self.params_update_layer_one(delta1, delta2, self.x[i])
            if j%500 == 0:
                print(f'Epoch: {j} Error: {e}')
    
    # Testavimo etapas
    def test(self, x_test):
        result = []
        for x in x_test:
            self.forward_propagation_layer_one(x)
            self.forward_propagation_layer_two()
            self.forward_propagation_layer_three()
            result.append(self.a3)
        return result